In [1]:
import pandas as pd
import requests
import urllib
import json

In [4]:
folder_url = 'https://disk.yandex.ru/d/s6wWqd8Ol_5IvQ'
file_url = 'data_test.csv'
url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download' + '?public_key=' + urllib.parse.quote(folder_url) + '&path=/' + urllib.parse.quote(file_url)


In [10]:
r = requests.get(url) # запрос ссылки на скачивание
h = json.loads(r.text)['href'] # 'парсинг' ссылки на скачивание

KeyError: 'href'

In [9]:
print(h)

{'message': 'Не удалось найти запрошенный ресурс.', 'description': 'Resource not found.', 'error': 'DiskNotFoundError'}


In [8]:
df = pd.read_csv(h, sep=';', error_bad_lines=False, comment='#') # загружаем данные

C:\Users\edelv\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


ValueError: Invalid file path or buffer object type: <class 'dict'>

In [ ]:
df.columns=['id', 'user1', 'user2']
df

In [ ]:
#pip install neo4j

In [ ]:
from neo4j import GraphDatabase

In [ ]:
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        if self.driver is not None:
            self.driver.close()

# Метод, который передает запрос в БД
    def query(self, query, db=None):
        assert self.driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.driver.session(database=db) if db is not None else self.driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

In [ ]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="Admin", password="12345678")
conn.query("CREATE OR REPLACE DATABASE graphdb")

In [ ]:
df.shape[0]

In [ ]:
duplicat = df[df.duplicated(['user1', 'user2'])]

В двнных нет посторяющихся значений

In [ ]:
for i in range(df.shape[0]):
    
    id = df.iloc[i]['id']
    user1 = df.iloc[i]['user1']
    user2 = df.iloc[i]['user2']
    str = '''
    MERGE (a: graphdb {user1:''' + "'" + user1 + "'})" + '''
    MERGE (b: graphdb {user2:''' + "'" + user2 + "'" + '''})
    CREATE (a)-[:SOLD]->(b)
    '''
    conn.query(str, db='graphdb')



In [ ]:
query_string = '''
MATCH
(n: graphdb)
RETURN n
'''
conn.query(query_string, db='graphdb')

In [ ]:
duble = df.query('user1 == "user2"')
duble

Взаимных пересечений нет. Закономерность отследить не предоставляется возможным.

По этому дальнейший анализ считаю не эффективным.
